### Notebook to generate DroneRF Features

In [1]:
import os
import numpy as np
from numpy import sum,isrealobj,sqrt
from numpy.random import standard_normal
import os
from sklearn.model_selection import train_test_split
from spafe.features.lfcc import lfcc
import spafe.utils.vis as vis
from scipy.signal import get_window
import scipy.fftpack as fft
from scipy import signal
import matplotlib.pyplot as plt
from datetime import date
from tqdm import tqdm

from loading_functions import *

In [2]:
def save_array(folder_path, feat, BI, DRONES, MODES, feat_name, seg_i):
    Xs_arr = np.array(feat)
    
    # labels
    y_bi_arr = np.array(BI)
    y_drones_arr = np.array(DRONES)
    y_modes_arr = np.array(MODES)

    data_save = {'feat': Xs_arr, 'bi': y_bi_arr, 'drones': y_drones_arr, 'modes': y_modes_arr}
#     return data_save

#     #Save data
    date_string = date.today()
    fp = folder_path+feat_name+"_"+str(n_per_seg)+"_"+str(seg_i)
    print(fp)
    np.save(fp, data_save)

In [3]:
# Dataset Info
main_folder = '/home/kzhou/Data/DroneRF/'
t_seg = 50
Xs_arr, ys_arr, y4s_arr, y10s_arr = load_dronerf_raw(main_folder, t_seg)
fs = 40e6 #40 MHz

rf_sig shape: 5
single rf_sig shape: (2, 2000000)
Xs len: 1
rf_sig shape: 5
single rf_sig shape: (2, 2000000)
Xs len: 2
rf_sig shape: 5
single rf_sig shape: (2, 2000000)
Xs len: 3
rf_sig shape: 5
single rf_sig shape: (2, 2000000)
Xs len: 4
rf_sig shape: 5
single rf_sig shape: (2, 2000000)
Xs len: 5
diff 5 file name: 00000L_13.csv
rf_sig shape: 5
single rf_sig shape: (2, 2000000)
Xs len: 6
rf_sig shape: 5
single rf_sig shape: (2, 2000000)
Xs len: 7
rf_sig shape: 5
single rf_sig shape: (2, 2000000)
Xs len: 8
rf_sig shape: 5
single rf_sig shape: (2, 2000000)
Xs len: 9
rf_sig shape: 5
single rf_sig shape: (2, 2000000)
Xs len: 10
rf_sig shape: 5
single rf_sig shape: (2, 2000000)
Xs len: 11
rf_sig shape: 5
single rf_sig shape: (2, 2000000)
Xs len: 12
rf_sig shape: 5
single rf_sig shape: (2, 2000000)
Xs len: 13
rf_sig shape: 5
single rf_sig shape: (2, 2000000)
Xs len: 14
rf_sig shape: 5
single rf_sig shape: (2, 2000000)
Xs len: 15
rf_sig shape: 5
single rf_sig shape: (2, 2000000)
Xs len: 16
r

rf_sig shape: 5
single rf_sig shape: (2, 2000000)
Xs len: 135
rf_sig shape: 5
single rf_sig shape: (2, 2000000)
Xs len: 136
rf_sig shape: 5
single rf_sig shape: (2, 2000000)
Xs len: 137
rf_sig shape: 5
single rf_sig shape: (2, 2000000)
Xs len: 138
rf_sig shape: 5
single rf_sig shape: (2, 2000000)
Xs len: 139
rf_sig shape: 5
single rf_sig shape: (2, 2000000)
Xs len: 140
rf_sig shape: 5
single rf_sig shape: (2, 2000000)
Xs len: 141
rf_sig shape: 5
single rf_sig shape: (2, 2000000)
Xs len: 142
rf_sig shape: 5
single rf_sig shape: (2, 2000000)
Xs len: 143
rf_sig shape: 5
single rf_sig shape: (2, 2000000)
Xs len: 144
rf_sig shape: 5
single rf_sig shape: (2, 2000000)
Xs len: 145
rf_sig shape: 5
single rf_sig shape: (2, 2000000)
Xs len: 146
rf_sig shape: 5
single rf_sig shape: (2, 2000000)
Xs len: 147
rf_sig shape: 5
single rf_sig shape: (2, 2000000)
Xs len: 148
rf_sig shape: 5
single rf_sig shape: (2, 2000000)
Xs len: 149
rf_sig shape: 5
single rf_sig shape: (2, 2000000)
Xs len: 150
rf_sig s

In [10]:
n_per_seg = 1024 # length of each segment (powers of 2)
win_type = 'hamming' # make ends of each segment match
high_low = 'H' #'L' # high or low range of frequency

# data saving folders
features_folder = '/home/kzhou/main/Features_DroneRF/'
date_string = date.today()
# folder naming: ARR_FEAT_NFFT_SAMPLELENGTH
arr_spec_folder = "ARR_SPEC_"+high_low+'_'+str(n_per_seg)+"_"+str(t_seg)+"/"
arr_psd_folder = "ARR_PSD_"+high_low+'_'+str(n_per_seg)+"_"+str(t_seg)+"/"

if high_low == 'H':
    i_hl = 0
elif high_low == 'L':
    i_hl = 1

In [11]:
# check if this set of parameters already exists
existing_folders = os.listdir(features_folder)
n_t_combo = high_low+'_'+str(n_per_seg)+"_"+str(t_seg)
if any([n_t_combo in s for s in existing_folders]):
    print('Features Already Exist - Do Not Generate')
else:
    os.mkdir(features_folder+arr_spec_folder)
    os.mkdir(features_folder+arr_psd_folder)

In [12]:
n_parts = 10 # process the data in 10 parts

indices_ranges = np.split(np.array(range(len(Xs_arr))), n_parts) 
for i in range(n_parts):
    BILABEL = []
    DRONELABEL = []
    MODELALBEL = []
    F_PSD = []
    F_SPEC = []
    ir = indices_ranges[i]
    for j in tqdm(range(len(ir))):
        d_real = Xs_arr[ir[j]][i_hl]
        # calculate PSD
        fpsd, Pxx_den = signal.welch(d_real, fs, window=win_type, nperseg=n_per_seg)
        F_PSD.append(Pxx_den)

        # calculate spectrogram
        fspec, t, Sxx = signal.spectrogram(d_real, fs, window=win_type, nperseg=n_per_seg)
        F_SPEC.append(Sxx)
        
        # Labels
        BILABEL.append(ys_arr[ir[j]])
        DRONELABEL.append(y4s_arr[ir[j]])
        MODELALBEL.append(y10s_arr[ir[j]])

    save_array(features_folder+arr_spec_folder, F_SPEC, BILABEL, DRONELABEL, MODELALBEL, 'SPEC', i)
    save_array(features_folder+arr_psd_folder, F_PSD, BILABEL, DRONELABEL, MODELALBEL, 'PSD', i)

100%|████████████████████████████████████████████████████████████████████████████████| 113/113 [00:09<00:00, 11.87it/s]


/home/kzhou/main/Features_DroneRF/ARR_SPEC_H_1024_50/SPEC_1024_0
/home/kzhou/main/Features_DroneRF/ARR_PSD_H_1024_50/PSD_1024_0


100%|████████████████████████████████████████████████████████████████████████████████| 113/113 [00:09<00:00, 12.01it/s]


/home/kzhou/main/Features_DroneRF/ARR_SPEC_H_1024_50/SPEC_1024_1
/home/kzhou/main/Features_DroneRF/ARR_PSD_H_1024_50/PSD_1024_1


100%|████████████████████████████████████████████████████████████████████████████████| 113/113 [00:09<00:00, 11.85it/s]


...

/home/kzhou/main/Features_DroneRF/ARR_SPEC_H_1024_50/SPEC_1024_7
/home/kzhou/main/Features_DroneRF/ARR_PSD_H_1024_50/PSD_1024_7


100%|████████████████████████████████████████████████████████████████████████████████| 113/113 [00:09<00:00, 12.20it/s]


/home/kzhou/main/Features_DroneRF/ARR_SPEC_H_1024_50/SPEC_1024_8
/home/kzhou/main/Features_DroneRF/ARR_PSD_H_1024_50/PSD_1024_8


100%|████████████████████████████████████████████████████████████████████████████████| 113/113 [00:09<00:00, 11.97it/s]


/home/kzhou/main/Features_DroneRF/ARR_SPEC_H_1024_50/SPEC_1024_9
/home/kzhou/main/Features_DroneRF/ARR_PSD_H_1024_50/PSD_1024_9
